In [7]:
# as extension task add the slot type
def create_slot(slot_name, slot_model):
    slot_model[slot_name] = "unfilled"

def fill_slot(slot_name, slot_model, slot_value):
    slot_model[slot_name] = slot_value

In [8]:
create_slot_func = {
    "name": "create_slot",
    "description": "function creates a slots based on a user query",
    "parameters": {
        "type": "object",
        "properties": {
            "slots_to_add": {
                "type": "array",
                "items": {
                    "type": "string",
                    "description": "name of the slot"
                },
                "description": "list of slots to add to accomplish what the user wants"
            }
        },
        "required": ["slot_name"]
    }
}

fill_slot_func = {
    "name": "fill_slot",
    "description": "function fills a slot name with slot value",
    "parameters": {
        "type": "object",
        "properties": {
            "slot_name":{
                "type": "string",
                "description": "name of the slot"
            },
            "slot_value":{
                "type": "string",
                "description": "value of the slot"
            }
        },
        "required": ["slot_name", "slot_value"]
    }
}

In [ ]:
from openai import OpenAI

analyze_system_prompt = '''
    You are an assistant that can determine whether to create or fill a slot.
    You are given a conversation context and a slot model.
    The slot model is a list of slot names that need to be filled. 
    A slot name corresponds to a piece of information that the user can provide.
    If the user answers with information answers a topic in the slot model then call fill_slot_func with the slot name and value.
    If the user asks a question or requests for information or an action that is not in the slot model then call create_slot_func and pass in a list of slot names designating the information the system would need answered by the user to accomplish what they request.
    Use the entire conversation context as context but only take an action based on the last user message.
'''

def analyze_response(convo_context, slot_model):
    tools_in = [
        {
            "type": "function",
            "function": create_slot_func,
        },
        {
            "type": "function",
            "function": fill_slot_func,
        }
    ]
    
    analyze_user_prompt = f'''
        Here is the conversation context:
        {convo_context}
        Here are the slots that need to be filled:
        {slot_model}
    '''
    
    msgs_in = [
        {"role": "system", "content": analyze_system_prompt},
        {"role": "user", "content": analyze_user_prompt}
    ]
        
    client = OpenAI()
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=msgs_in,
        tools=tools_in,
    )
    
    tool_choice = completion.choices[0].message.tool_calls[0] if completion.choices[0].message.tool_calls else None
    print("tool_choice", tool_choice)
    

In [13]:
slot_model_in = "departure_date, budget, return_date, preferred_airline"
convo_context_in = '''
    System: Hello how may I assist you in booking a flight today?
    User: I would like to book a flight to New York.
    System: What is your departure city?
    User: I am departing from Los Angeles.
    System: What is your budget?
    User: My budget is $500.
'''

analyze_response(convo_context_in, slot_model_in)

tool_choice ChatCompletionMessageToolCall(id='call_Ko5wGGnIA0jbjoovnTdtUkBE', function=Function(arguments='{"slot_name":"budget","slot_value":"$500"}', name='fill_slot'), type='function')


In [ ]:
slot_model_in = "preferred_airline"
convo_context_in = '''
    System: Hello how may I assist you in booking a flight today?
    User: I would like to book a flight to New York.
    System: What is your departure city?
    User: I am departing from Los Angeles.
    System: What is your budget?
    User: My budget is $500.
    System: When is your departure date?
    User: I am departing on the 10th of November.
    System: What is your return date?
    User: I am returning on the 15th.
    System: Is there anything else you would like to add?
    User: Are you able to book a hotel for me?
'''

analyze_response(convo_context_in, slot_model_in)

tool_choice ChatCompletionMessageToolCall(id='call_wP9wQ0luunRFidXNn8vO9Nxe', function=Function(arguments='{"slots_to_add":["hotel_location","check_in_date","check_out_date","hotel_rating"]}', name='create_slot'), type='function')
